In [157]:
"""For altering memory usage"""
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
import time
n = get_gap_memory_pool_size()
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()

In [18]:
"""List of indices of small groups by GAP library: [x,y]."""
small = []
for i in range(607,608):
    if i not in [511,575]:
        small += gap.IdsOfAllSmallGroups(i+1)
#         print(i+1)
    else:
        print(i+1)
print(len(small))

195


In [88]:
"""For new problem with general subgroups."""
class induce():
    """Function to return list of class sizes in default order."""
    def __init__(self):
        pass

    """Returns class sizes up to conjugacy"""
    def class_sizes(self, G):
        group_order = G.order()
        reps = G.conjugacy_classes_representatives()
        class_sizes = []
        for g in reps:
            class_sizes.append(group_order / G.centralizer(g).order())
#         print(class_sizes)
#         orders = []
#         for rep in reps:
#             orders.append(rep.order())
#         print(orders)
        return class_sizes

    """Function to find all subgroups. Returns list of subgroups."""
    def subgroups(self, G):
        subgroups = G.conjugacy_classes_subgroups()
#         print(subgroups)
        return subgroups

    """Function to find induced trivial character of each subgroup. Returns a list of lists."""
    def get_induced(self, subgroups, G):
        ind = []
        for subgroup in subgroups:
            ind.append(subgroup.trivial_character().induct(G))
#         print(ind)
        return ind

    """Forms the matrices that we use to solve the simultaneous equation later. The last entry is the trivial representation."""
    def mat(self, induced):
        matrix = []
        for char in induced:
            matrix.append(list(char.values()))
        neg = []
        for i in range(len(matrix[0])):
            neg.append(-1)
        matrix[-1] = neg
#         for line in matrix:
#             print(line)
        return matrix

    """Check if a subgroup H of G is core-free"""
    def core_free(self, G, H):
        if H.order() == 1: 
            return True
        core = self.subgroups(H)[1:]
        for subsub in core:
            if subsub.is_normal(G) == True:
                return False
        return True

    """Find all subgroups of G containing H (just the conjugates)"""
    def subgroups_containing(self, G, H, subgroups):
        subgroups_list = [] 
        h = H.order()
        H_conjugates = []
        H_normalizer = G.normalizer(H)
        H_norm_cosets = G.cosets(H_normalizer)
        for coset in H_norm_cosets:
            g = coset[0]
            H_conjugates.append(H.conjugate(g))
        for subgroup in subgroups:
            if subgroup.order() > h:
                sub = False
                i = len(H_conjugates)
                while sub == False and i != 0:
                    sub = H_conjugates[i-1].is_subgroup(subgroup)
                    i -= 1
                if sub == True:
                    subgroups_list.append(subgroup)
        return subgroups_list
    
    """Finding PHG"""
    def get_phg(self, G, subgroup,j):
        gens = []
        gens += subgroup.list()
        X = []
        elems = G.list()
        for g in elems:
            for h in subgroup.conjugate(g).list():
                if h not in X:
                    X.append(h)
        for g in elems:
            if g**j not in X:
                gens.append(g)
        hp_group = PermutationGroup(gens)
        print("Hp" + str(gap.IdSmallGroup(hp_group)), str(gap.IdSmallGroup(subgroup)))
            

    """Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise.
    phg option to test hypothesis
    """
    def test(self, G):
        """Filter abelian groups"""
        center = G.center()
        gOrder = G.order() 
        if center.order() == gOrder:
            j = 2
            for elem in center.list():
                j = max(j, elem.order())
                if is_prime(j) != True:
                    return ""
            if is_prime(gOrder) == True:
                return ""
            else:
                return "1,1  "

        """Begin of main test"""
        a = self.class_sizes(G)
        b = self.subgroups(G)
        zeros = [0]*len(a)
        ret_list = ""
        for i in range(1,len(b)):
            cor = self.core_free(G, b[i])
            if cor == True:
                try:
#                     """Finds P_H(G)"""
#                     self.get_phg(G,b[i],j)

                    b_i = self.subgroups_containing(G, b[i], b)
                    c = self.get_induced(b_i, G)
                    d_i = self.mat(c)
                    e = Matrix(d_i).T
                    temp = b[i].trivial_character().induct(G).values()
                    Y = vector(temp)
                    p = MixedIntegerLinearProgram(solver='ppl')
                    w = p.new_variable(nonnegative=True)
                    p.add_constraint(e*w == Y)
                    p.solve()
                    
                    """Finds the decomposition structure."""
                    subgroupsH_i = []
                    coeffs = []
#                     print("Induced char of H: " + str(temp))
                    out = []
#                     print("Subgroups in decomposition")
                    for j, v in (p.get_values(w).items()):
                        out.append('w_%s = %s' % (j, v))
                        """Prints subgroup and subgroup order."""
                        if round(v,6) != 0:
#                             print(e.T[j])
#                             print("Coef:" + str(v), "Order:" + str(gap.IdSmallGroup(b_i[j])))
                            subgroupsH_i.append(b_i[j])
                            coeffs.append(v)
#                     print(e.T)
#                     print(out)

                    """Prints the solution and SmallGroup Id"""
                    if b[i].order() != 1:
                        criteria_4 = self.criteria_4(G, subgroupsH_i, b[i], coeffs)
#                         criteria_2 = self.criteria_2(G, subgroupsH_i)
#                         criteria_1 = self.criteria_1(G, subgroupsH_i, b[i], coeffs)
                    if criteria_4:
                        ret_list += str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2])+"  "
                        print("Pair G-H:" + str(gap.IdSmallGroup(G)[1]) + "," + str(gap.IdSmallGroup(G)[2]) +"    "+ str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2]))
                except:
                    pass
#                     print('fail')
        return ret_list
        
    def criteria_1(self, G, H_i, H, coeffs):
        """H_i is the list of groups. This criteria is when H is in complement"""
        largest = H_i[-2]
        if not largest.is_normal(G):
            return False
        normals = []
        subgroupsOfLargest = self.subgroups(largest)
        for subgroup in subgroupsOfLargest:
            if subgroup.order() != 1 and subgroup.is_normal(largest) == True:
                #Check if its a possible candidate
                if (largest.order()/H.order())%subgroup.order() == 0:
                    #Find size of intersection
                    if subgroup.intersection(H).order() == 1:
                        normals.append(subgroup)
        
#         for normal in normals:
#             print(gap.Order(normal))
        
        if len(normals) == 0:
            print("Fails Criteria 1")
            return False
        
        originalH_minus2 = H_i[-2]
        
        for normal in normals:
            #Next we continue to check 
            try:
                H_i[-2] = PermutationGroup(gap_group = gap.ComplementClassesRepresentatives(originalH_minus2,normal)[1])
            except:
                continue 
                
            complements = []
            try:
                complements = gap.ComplementClassesRepresentatives(G,normal)
            except:
                continue
                
            complementH = ""
            for complement in complements:
                for candidateComp in gap.ConjugateSubgroups(G,complement):
                    candidate = True
                    for sub in H_i[:-1]:
                        if not gap.IsSubgroup(candidateComp,sub):
                            candidate = False
                            break
                    if candidate:
                        complementH = candidateComp
                        break
                if complementH != "":
                    break
                
            if complementH == "":
                continue
            else:    
                inducedTrivials = []
                complementH = PermutationGroup(gap_group = complementH)
                
                for sub in H_i[:-1]:
                    induce_sub_to_complementH = sub.trivial_character().induct(complementH).values()
                    inducedTrivials.append(induce_sub_to_complementH)
                
                induce_H_to_complementH = H.trivial_character().induct(complementH).values()
                inducedTrivials.append(induce_H_to_complementH)
                
#                 print(inducedTrivials)

                sumCheck = []
                for zero in range(len(induce_H_to_complementH)):
                    sumCheck.append(-coeffs[-1])
                sumCheck = vector(sumCheck)
                
                for summand in range(len(inducedTrivials)-1):
                    sumCheck += coeffs[summand] * vector(inducedTrivials[summand])
                    
#                 print(sumCheck)
#                 print(sumCheck == vector(inducedTrivials[-1]))
                if sumCheck == vector(inducedTrivials[-1]):
                    return True
                
        #Check 2 where we directly construct a decomposition. Will make if necessary
        return False
        
    def criteria_2(self, G, H_i):
        """S-partitions"""
        for g in G.list():
            found = False
            for H in H_i[:-1]:
                if g in H.list():
                    found = True
                    break
            if found:
                continue
            else:
                return False
        return True
    
    def criteria_3(self, G, H_i, H, coeffs):
        """H is in kernel
        """
        gap.NormalSubgroups(H_i[-2])
        normalClosure = PermutationGroup(gap_group = gap.NormalClosure(G,H))
        print(gap.Order(normalClosure))
        possibleNormals = []
        for candidate in gap.NormalSubgroups(H_i[-2]):
            if gap.IsSubgroup(candidate,normalClosure):
                possibleNormals.append(candidate)
        possibleNormals.reverse()
        
        for normalClosure in possibleNormals:
            try:
                complements = gap.ComplementClassesRepresentatives(G,normalClosure)
            except:
                continue
                
            for complement in complements:
                H_iNew = []
                for i in range(len(H_i)-2):
                    tempClosure = gap.ClosureGroup(H_i[i],normalClosure)
                    foundRep = False
                    if gap.Order(tempClosure) != gap.Order(H_i[i])*gap.Order(normalClosure)/gap.Order(H) and not gap.IsNormal(tempClosure,normalClosure):
                        break
                    for comp in gap.ComplementClassesRepresentatives(tempClosure,normalClosure):
                        for candidateComp in gap.ConjugateSubgroups(G,comp):
                            if gap.IsSubgroup(complement, candidateComp):
                                H_iNew.append(candidateComp)
                                foundRep = True
                                break
                        if foundRep:
                            break
                    if not foundRep:
                        break
                if not foundRep:
                    continue
        #             print(gap.Order(H_i[i]))
        
                foundRep = False
                try: 
                    reps = gap.ComplementClassesRepresentatives(H_i[-2],normalClosure)
                except:
                    continue
                for comp in reps:
                    for candidateComp in gap.ConjugateSubgroups(G,comp):
                        if gap.IsSubgroup(complement, candidateComp):
                            H_iNew.append(candidateComp)
                            foundRep = True
                            break
                    if foundRep:
                        break
                if not foundRep:
                    continue 
                    
                gens = []
                for sub in H_iNew:
                    gens += gap.GeneratorsOfGroup(sub)
                complementH = PermutationGroup(gap_group = gap.Group(gens))
        #         print(complementH.order())

                if complementH.order() * gap.Order(normalClosure) != G.order():
                    continue

                else:    
                    inducedTrivials = []

                    for sub in H_iNew:
                        sub2 = complementH.subgroup(PermutationGroup(gap_group = sub))
                        induce_sub_to_complementH = sub2.trivial_character().induct(complementH).values()
                        inducedTrivials.append(induce_sub_to_complementH)

                    trivialComplementH = PermutationGroup([]).trivial_character().induct(complementH).values()
                    inducedTrivials.append(trivialComplementH)

                    print(inducedTrivials)

                    sumCheck = []
                    for zero in range(len(trivialComplementH)):
                        sumCheck.append(-coeffs[-1])
                    sumCheck = vector(sumCheck)

                    for summand in range(len(inducedTrivials)-1):
                        sumCheck += coeffs[summand] * vector(inducedTrivials[summand])

                    print(sumCheck)
                    print(sumCheck == vector(inducedTrivials[-1]))
                    if sumCheck == vector(inducedTrivials[-1]):
                        return True
        return False       
    
    def criteria_4(self, G, H_i, H, coeffs):
        """H is in kernel and complement
        """
        subOfHs = gap.NormalSubgroups(H)
        for subOfH in subOfHs:
            if gap.Order(subOfH) == 1:
                continue
            normalClosure = PermutationGroup(gap_group = gap.NormalClosure(G,subOfH))
            print("normalClosure Order " + str(gap.Order(normalClosure)))
            possibleNormals = []
            for candidate in gap.NormalSubgroups(H_i[-2]):
                if gap.IsSubgroup(candidate,normalClosure):
                    possibleNormals.append(candidate)
            possibleNormals.reverse()

            for normalClosure in possibleNormals:
                try:
                    complements = gap.ComplementClassesRepresentatives(G,normalClosure)
                except:
                    continue

                for complement in complements:
                    H_iNew = []
                    for i in range(len(H_i)-2):
                        tempClosure = gap.ClosureGroup(H_i[i],normalClosure)
                        foundRep = False
                        if gap.Order(tempClosure) != gap.Order(H_i[i])*gap.Order(normalClosure)/gap.Order(subOfH) and not gap.IsNormal(tempClosure,normalClosure):
                            break
#                         try: 
#                             g = gap.ComplementClassesRepresentatives(tempClosure,normalClosure)
#                         except:
#                             print("Not split")
                        for comp in gap.ComplementClassesRepresentatives(tempClosure,normalClosure):
                            for candidateComp in gap.ConjugateSubgroups(G,comp):
                                if gap.IsSubgroup(complement, candidateComp):
                                    H_iNew.append(candidateComp)
                                    foundRep = True
                                    break
                            if foundRep:
                                break
                        if not foundRep:
                            break
#                         print(gap.Order(H_i[i]))
                    if not foundRep:
                        continue

                    foundRep = False
                    try: 
                        reps = gap.ComplementClassesRepresentatives(H_i[-2],normalClosure)
                    except:
                        continue
                    for comp in reps:
                        for candidateComp in gap.ConjugateSubgroups(G,comp):
                            if gap.IsSubgroup(complement, candidateComp):
                                H_iNew.append(candidateComp)
                                foundRep = True
                                break
                        if foundRep:
                            break
                    if not foundRep:
                        continue 

                    gens = []
                    for sub in H_iNew:
                        gens += gap.GeneratorsOfGroup(sub)
                    complementH = PermutationGroup(gap_group = gap.Group(gens))
                    print(gap.Order(normalClosure), complementH.order())

                    if complementH.order() * gap.Order(normalClosure) != G.order():
                        continue

                    else:    
                        inducedTrivials = []

                        for sub in H_iNew:
                            sub2 = complementH.subgroup(PermutationGroup(gap_group = sub))
                            induce_sub_to_complementH = sub2.trivial_character().induct(complementH).values()
                            inducedTrivials.append(induce_sub_to_complementH)
                        
                        try:
                            HinComplement = PermutationGroup(gap_group = gap.ComplementClassesRepresentatives(H,subOfH)[1])
                            print("Success")
                        except:
                            print("Nope")
                            continue
                        HinComplement = complementH.subgroup(HinComplement)
                        HinCompInd = HinComplement.trivial_character().induct(complementH).values()
                        inducedTrivials.append(HinCompInd)

                        print(inducedTrivials)

                        sumCheck = []
                        for zero in range(len(HinCompInd)):
                            sumCheck.append(-coeffs[-1])
                        sumCheck = vector(sumCheck)

                        for summand in range(len(inducedTrivials)-1):
                            sumCheck += coeffs[summand] * vector(inducedTrivials[summand])

                        print(sumCheck)
                        print(sumCheck == vector(inducedTrivials[-1]))
                        if sumCheck == vector(inducedTrivials[-1]):
                            return True
        return False       

In [94]:
a = "588,33 588,34 600,148 600,149 600,150"
a = a.split(" ")
small = []
for i in range(len(a)):
    if a[i] == '':
        continue
    a[i] = a[i].split(",")
    small.append([int(a[i][0]),int(a[i][1])])
print(small)

[[588, 33], [588, 34], [600, 148], [600, 149], [600, 150]]


In [92]:
"""Main function
If group is in id form.
https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
This works on CoCalc/non-windows system with gap_packages library installed."""
if __name__ == "__main__":
    inducer = induce()
    groups = [[500, 23], [588, 33], [588, 34], [600, 148], [600, 149], [600, 150], [600, 152]]
    out_a = []
    out_b = []
    for groupid in groups:
        G = gap.SmallGroup(groupid)
        ret = inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
        if len(ret) > 0:
            out_a.append(groupid)
            out_b.append(ret)
#         try:
#             inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
#         except:
#             pass
    for a in out_a:
        print(str(a[0])+","+str(a[1]))
    for b in out_b:
        print(b)
    """If group is in SAGE form.
    https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups"""
#     C5 = CyclicPermutationGroup(5)
#     G = C5.holomorph()
#     C2 = CyclicPermutationGroup(2)
#     groups = [direct_product_permgroups([C2,G])]
#     for group in groups:
#         try:
#             result = inducer.test(group)
#         except:
#             pass

normalClosure Order 250
normalClosure Order 250
normalClosure Order 25
25 20
Success
[[5, 1, 1, 0, 1], [2, 0, 2, 2, 0], [10, 0, 2, 0, 0]]
(10, 0, 2, 0, 0)
True
Pair G-H:500,23    10,1
normalClosure Order 250
normalClosure Order 25
25 20
Success
[[5, 1, 1, 0, 1], [2, 0, 2, 2, 0], [10, 0, 2, 0, 0]]
(10, 0, 2, 0, 0)
True
Pair G-H:500,23    10,1
normalClosure Order 250
normalClosure Order 25
25 20
Success
[[5, 1, 1, 0, 1], [2, 0, 2, 2, 0], [10, 0, 2, 0, 0]]
(10, 0, 2, 0, 0)
True
Pair G-H:500,23    10,1
normalClosure Order 250
normalClosure Order 25
25 20
Success
[[5, 1, 1, 0, 1], [2, 0, 2, 2, 0], [10, 0, 2, 0, 0]]
(10, 0, 2, 0, 0)
True
Pair G-H:500,23    10,1
normalClosure Order 250
normalClosure Order 25
25 20
Success
[[5, 1, 1, 0, 1], [2, 0, 2, 2, 0], [10, 0, 2, 0, 0]]
(10, 0, 2, 0, 0)
True
Pair G-H:500,23    10,1
normalClosure Order 250
normalClosure Order 25
25 20
Success
[[5, 1, 1, 0, 1], [2, 0, 2, 2, 0], [10, 0, 2, 0, 0]]
(10, 0, 2, 0, 0)
True
Pair G-H:500,23    10,1
normalClosure Or

In [93]:
for a in out_a:
    print(str(a[0])+","+str(a[1]))
for b in out_b:
    print(b)

500,23
588,33
588,34
600,148
600,149
600,150
600,152
10,1  10,1  10,1  10,1  10,1  10,1  
3,1  6,2  
4,1  
3,1  6,2  12,2  
3,1  6,2  8,1  12,2  
8,4  
6,2  10,1  10,1  
